In [ ]:
%run -p train_cnf_tol.py --data mnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 8000 --test_batch_size 8000 --save experiments/cnf_nocond_bs8K_100timeslr_exploretol2_resume --conditional False --lr 0.001 --warmup_iters 113 --atol 1e-4  --rtol 1e-4 --resume experiments/cnf_nocond_bs8K_100timeslr_exploretol2/checkpt.pth

/tancode/repos/tan-ffjord/train_cnf_exploretol2.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.odenvp_exploretol2 as odenvp
import lib.multiscale_parallel as multiscale_parallel

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
parser = argparse.ArgumentParser("Continuous Normalizing Flow")
parser.add_argument("--data", choices=["mnist", "svhn", "cifar10", 'lsun_church'], typ

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 0007 | Time 34.3206(65.3586) | Bit/dim 1.1095(1.1103) | Steps 446(446.16) | Grad Norm 0.8238(1.0192) | Total Time 10.00(10.00)
Iter 0008 | Time 31.7537(64.3505) | Bit/dim 1.1077(1.1102) | Steps 446(446.15) | Grad Norm 0.7234(1.0103) | Total Time 10.00(10.00)
Iter 0009 | Time 33.6701(63.4301) | Bit/dim 1.1138(1.1103) | Steps 452(446.33) | Grad Norm 0.6347(0.9991) | Total Time 10.00(10.00)
Iter 0010 | Time 32.3198(62.4968) | Bit/dim 1.1048(1.1102) | Steps 446(446.32) | Grad Norm 0.5280(0.9849) | Total Time 10.00(10.00)
Iter 0011 | Time 33.0271(61.6127) | Bit/dim 1.1047(1.1100) | Steps 452(446.49) | Grad Norm 0.4278(0.9682) | Total Time 10.00(10.00)
Iter 0012 | Time 32.6688(60.7444) | Bit/dim 1.1041(1.1098) | Steps 446(446.48) | Grad Norm 0.3305(0.9491) | Total Time 10.00(10.00)
Iter 0013 | Time 32.1377(59.8862) | Bit/dim 1.1096(1.1098) | Steps 446(446.46) | Grad Norm 0.2631(0.9285) | Total Time 10.00(10.00)
validating...
Epoch 00

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 0014 | Time 32.3646(59.0605) | Bit/dim 1.1057(1.1097) | Steps 446(446.45) | Grad Norm 0.2335(0.9076) | Total Time 10.00(10.00)
Iter 0015 | Time 33.3512(58.2892) | Bit/dim 1.1091(1.1097) | Steps 446(446.43) | Grad Norm 0.2512(0.8880) | Total Time 10.00(10.00)
Iter 0016 | Time 32.9818(57.5300) | Bit/dim 1.1069(1.1096) | Steps 446(446.42) | Grad Norm 0.2955(0.8702) | Total Time 10.00(10.00)
Iter 0017 | Time 31.5632(56.7510) | Bit/dim 1.1115(1.1096) | Steps 446(446.41) | Grad Norm 0.3503(0.8546) | Total Time 10.00(10.00)
Iter 0018 | Time 32.9020(56.0355) | Bit/dim 1.1083(1.1096) | Steps 446(446.40) | Grad Norm 0.4027(0.8410) | Total Time 10.00(10.00)
Iter 0019 | Time 32.4863(55.3291) | Bit/dim 1.0990(1.1093) | Steps 446(446.38) | Grad Norm 0.4736(0.8300) | Total Time 10.00(10.00)
Iter 0020 | Time 31.9967(54.6291) | Bit/dim 1.1055(1.1092) | Steps 446(446.37) | Grad Norm 0.4827(0.8196) | Total Time 10.00(10.00)
validating...
Epoch 00

...

===> Using batch size 8000. Total 7 iterations/epoch.
Iter 4431 | Time 34.2798(34.5068) | Bit/dim 1.0358(1.0516) | Steps 476(473.98) | Grad Norm 2.4485(4.2419) | Total Time 10.00(10.00)
Iter 4432 | Time 34.0089(34.4919) | Bit/dim 1.0391(1.0512) | Steps 476(474.04) | Grad Norm 0.6901(4.1353) | Total Time 10.00(10.00)
Iter 4433 | Time 34.8270(34.5020) | Bit/dim 1.0365(1.0507) | Steps 488(474.46) | Grad Norm 2.4490(4.0847) | Total Time 10.00(10.00)
Iter 4434 | Time 35.3523(34.5275) | Bit/dim 1.0392(1.0504) | Steps 476(474.50) | Grad Norm 2.4215(4.0348) | Total Time 10.00(10.00)
Iter 4435 | Time 35.2068(34.5478) | Bit/dim 1.0358(1.0500) | Steps 476(474.55) | Grad Norm 0.5733(3.9310) | Total Time 10.00(10.00)
Iter 4436 | Time 35.0681(34.5635) | Bit/dim 1.0397(1.0496) | Steps 482(474.77) | Grad Norm 3.5009(3.9181) | Total Time 10.00(10.00)
Iter 4437 | Time 35.7299(34.5984) | Bit/dim 1.0391(1.0493) | Steps 482(474.99) | Grad Norm 3.0943(3.8934) | Total Time 10.00(10.00)
validating...
Epoch 06

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 4438 | Time 34.2008(34.5865) | Bit/dim 1.0419(1.0491) | Steps 482(475.20) | Grad Norm 1.0963(3.8095) | Total Time 10.00(10.00)
Iter 4439 | Time 34.4884(34.5836) | Bit/dim 1.0391(1.0488) | Steps 482(475.40) | Grad Norm 5.1079(3.8484) | Total Time 10.00(10.00)
Iter 4440 | Time 34.7925(34.5898) | Bit/dim 1.0415(1.0486) | Steps 488(475.78) | Grad Norm 4.1684(3.8580) | Total Time 10.00(10.00)
Iter 4441 | Time 35.8106(34.6265) | Bit/dim 1.0339(1.0481) | Steps 488(476.15) | Grad Norm 1.8056(3.7964) | Total Time 10.00(10.00)
Iter 4442 | Time 34.9220(34.6353) | Bit/dim 1.0432(1.0480) | Steps 488(476.50) | Grad Norm 8.0273(3.9234) | Total Time 10.00(10.00)
Iter 4443 | Time 36.0602(34.6781) | Bit/dim 1.0410(1.0478) | Steps 482(476.67) | Grad Norm 6.1353(3.9897) | Total Time 10.00(10.00)
Iter 4444 | Time 34.2267(34.6645) | Bit/dim 1.0405(1.0476) | Steps 482(476.83) | Grad Norm 4.1565(3.9947) | Total Time 10.00(10.00)
validating...
Epoch 06

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 4445 | Time 35.8829(34.7011) | Bit/dim 1.0515(1.0477) | Steps 482(476.98) | Grad Norm 12.9837(4.2644) | Total Time 10.00(10.00)
Iter 4446 | Time 33.7179(34.6716) | Bit/dim 1.0495(1.0477) | Steps 452(476.23) | Grad Norm 6.9360(4.3445) | Total Time 10.00(10.00)
